In [23]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/utils")
from utils import SimpleGraphVoltDatasetLoader_Lazy
from torch_geometric_temporal.signal import temporal_signal_split
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN
from tqdm import tqdm

In [24]:
class avgModel():
    def __init__(self, num_timesteps_out, voltage_index):
        self.num_timesteps_out = num_timesteps_out
        self.voltage_index = voltage_index

    def __call__(self, x, edge_index):
        predicted_voltages = torch.mean(x[:, self.voltage_index, :], dim=1, keepdim=True)
        return predicted_voltages.repeat(1, self.num_timesteps_out)

    def train(self):
        pass

    def eval(self):
        pass

In [25]:
trafo_id = "T1330"
epochs = 25
num_timesteps_in = 12
num_timesteps_out = 4
train_ratio = 0.7
test_ratio_vs_eval_ratio = 0.5
learning_rate = 0.01
device_str = 'cpu'

In [26]:
#get dateime string of now
now = pd.Timestamp.now().strftime("%Y%m%d%H%M%S")

In [27]:
print("Loading data...", end="")
loader = SimpleGraphVoltDatasetLoader_Lazy(trafo_id, num_timesteps_in, num_timesteps_out)
print(" done")
loader_data_index = loader.snapshot_index
# loader_data = loader.get_dataset(num_timesteps_in=num_timesteps_in, num_timesteps_out=num_timesteps_out)

Loading data...Voltage index: 5
 done


In [28]:
train_dataset, test_eval_dataset = loader.temporal_signal_split_lazy(loader_data_index, train_ratio)
test_dataset, eval_dataset = loader.temporal_signal_split_lazy(test_eval_dataset, test_ratio_vs_eval_ratio)

In [29]:
device = torch.device(device_str)
model = avgModel(num_timesteps_out, loader.voltage_index)
loss_fn = torch.nn.L1Loss

In [30]:
def run_avg_loss(dataset, loader, loss_fn, device):
    loss_run_avg = 0
    for snapshot_i in dataset:
        #snapshot.to(device)
        snapshot = loader.get_snapshot(snapshot_i)
    
        out = model(snapshot.x, snapshot.edge_index)
        loss = loss_fn()(out, snapshot.y)
        loss_run_avg += loss.item()

    return loss_run_avg

print(run_avg_loss(train_dataset, loader, loss_fn, device))

        

10771.335520049557
